In [53]:
import numpy as np
import pandas as pd
import os

In [112]:
def scale(x):
    return (x - x.mean())/x.std()

def winsor(x, floor, root): 
    x[x < x.quantile(floor)] = x.quantile(floor)
    x[x > x.quantile(root)] = x.quantile(root)
    return x

def type_change(x):
    res = []
    for i in x:
        try:
            res.append(float(i))
        except (ValueError):
            res.append(np.nan)
    return res

In [3]:
WindA = pd.read_excel('wind全A行情.xlsx', parse_dates = True)
WindA = WindA.iloc[:WindA.shape[0]-2,:]
WindA.columns = list(map(lambda x: x.split(sep="\n")[0],WindA.columns.tolist()))
WindA.set_index("时间",inplace=True)

In [25]:
WindA_v1 = pd.DataFrame()
WindA_v1["涨跌幅"] = (WindA["收盘价"] - WindA["前收盘价"])/WindA["前收盘价"]
WindA_v1[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']] = WindA[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']]
WindA_v1 = WindA_v1.apply(winsor,floor=0.025,root=0.975,axis = 0)
WindA_v1 = WindA_v1.apply(scale,axis=0)
WindA_v1.columns = "WindA_" + WindA_v1.columns

## 工业行情

In [17]:
Industry = pd.read_excel('工业行情.xlsx', parse_dates = True)
Industry = Industry.iloc[:Industry.shape[0]-2,:]
Industry.columns = list(map(lambda x: x.split(sep="\n")[0],Industry.columns.tolist()))
Industry.set_index("时间",inplace=True)

In [26]:
Industry_v1 = pd.DataFrame()
Industry_v1["涨跌幅"] = (Industry["收盘价"] - Industry["前收盘价"])/Industry["前收盘价"]
Industry_v1[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']] = Industry[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']]
Industry_v1 = Industry_v1.apply(winsor,floor=0.025,root=0.975,axis = 0)
Industry_v1 = Industry_v1.apply(scale,axis=0)
Industry_v1.columns = "Industry_" + Industry_v1.columns

## 信息技术

In [27]:
info_tech = pd.read_excel('工业行情.xlsx', parse_dates = True)
info_tech = info_tech.iloc[:info_tech.shape[0]-2,:]
info_tech.columns = list(map(lambda x: x.split(sep="\n")[0],info_tech.columns.tolist()))
info_tech.set_index("时间",inplace=True)

In [28]:
info_tech_v1 = pd.DataFrame()
info_tech_v1["涨跌幅"] = (info_tech["收盘价"] - info_tech["前收盘价"])/info_tech["前收盘价"]
info_tech_v1[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']] = info_tech[['收盘价','成交量', '换手率', '周最高价',
       '周最低价', '周均价', '周成交量', '周涨跌幅', '自由流通股本', '市盈率PE(TTM)',
       '市净率PB(LF)', '股息率(近12个月)', '一致预测净利润(FY1)', '一致预测净利润(FY2)']]
info_tech_v1 = info_tech_v1.apply(winsor,floor=0.025,root=0.975,axis = 0)
info_tech_v1 = info_tech_v1.apply(scale,axis=0)
info_tech_v1.columns = "info_tech_" + info_tech_v1.columns

## 宏观变量

In [30]:
macro_var = pd.read_excel('宏观变量.xlsx', parse_dates = True)

In [36]:
macro_var.columns = ['时间', 'CPI', 'PPI', '出口价格指数(总)',
       '出口价格指数(车船航)', '进口价格指数(总)',
       '进口价格指数(车船航)', '固定资产投资完成额', '社会消费品零售总额',
       'M2', '金融机构各项贷款余额', '社会融资规模']
       
macro_var.set_index('时间',inplace=True)
macro_var.index = macro_var.index.strftime("%Y-%m")
macro_var.columns = "宏观_" + macro_var.columns

In [47]:
temp = info_tech_v1
temp["date_trans"] = info_tech_v1.index.strftime("%Y-%m")
info_macro = pd.merge(temp,macro_var,left_on="date_trans",right_index=True)
del info_macro["date_trans"]

In [49]:
temp = Industry_v1
temp["date_trans"] = Industry_v1.index.strftime("%Y-%m")
industry_macro = pd.merge(temp,macro_var,left_on="date_trans",right_index=True)
del industry_macro["date_trans"]

In [52]:
info_macro.to_excel("data_prepocessed/信息行业和宏观.xlsx")
industry_macro.to_excel("data_prepocessed/工业行业和宏观.xlsx")

### 个股

In [54]:
os.listdir("成长")

['002415.SZ.xlsx', '002475.SZ.xlsx', '601012.SH.xlsx']

In [ ]:
temp = pd.read_excel("成长/"+,parse_dates=True)

In [ ]:
os.listdir("成长")

In [179]:
## 成长
for i, stk in enumerate(os.listdir("成长")):
    temp = pd.read_excel("成长/"+stk,parse_dates=True)
    temp = temp.iloc[:temp.shape[0]-2,:]
    temp.columns = list(map(lambda x: x.split(sep="\n")[0],temp.columns.tolist()))
    temp.set_index("日期",inplace=True)
    stk_name = temp["简称"][-1]
    temp.drop(temp.loc[:,"代码":"简称"].columns, axis = 1, inplace=True)
    del temp["B股流通市值(元)"]
    ## 因变脸
    ## 未来一期价格
    Y = pd.Series(temp["收盘价(元)"][1:].tolist(), index = list(temp.index)[:-1],name="股价")
    pd.merge(Y,temp,left_index=True,right_index=True,how="left")

    ## 类型转化
    for column in temp.columns:
        if temp[column].dtype == object:
            temp[column] = type_change(temp[column])
        temp[column].astype(float)

    ## 缺失值填充
    if temp.isnull().sum().sum() == 0:
        pass
    else:
        temp.apply(lambda x: x.fillna(x.mean()))

    temp = temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
    temp= temp.apply(scale,axis=0)

    temp = pd.merge(Y,temp,left_index=True,right_index=True,how="left")
    res = pd.merge(temp, industry_macro,left_index=True,right_index=True,how="left").fillna(method="ffill")
    res = res.dropna(axis = 0)
    res.to_excel("data_prepocessed/"+stk_name+"_"+stk[:6]+"_稳定.xlsx")

In [180]:
## 稳定
for i, stk in enumerate(os.listdir("稳定")):
    temp = pd.read_excel("稳定/"+stk,parse_dates=True)
    temp = temp.iloc[:temp.shape[0]-2,:]
    temp.columns = list(map(lambda x: x.split(sep="\n")[0],temp.columns.tolist()))
    temp.set_index("日期",inplace=True)
    stk_name = temp["简称"][-1]
    temp.drop(temp.loc[:,"代码":"简称"].columns, axis = 1, inplace=True)
    del temp["B股流通市值(元)"]
    ## 因变脸
    ## 未来一期价格
    Y = pd.Series(temp["收盘价(元)"][1:].tolist(), index = list(temp.index)[:-1],name="股价")
    pd.merge(Y,temp,left_index=True,right_index=True,how="left")

    ## 类型转化
    for column in temp.columns:
        if temp[column].dtype == object:
            temp[column] = type_change(temp[column])
        temp[column].astype(float)

    ## 缺失值填充
    if temp.isnull().sum().sum() == 0:
        pass
    else:
        temp.apply(lambda x: x.fillna(x.mean()))

    temp = temp.apply(winsor,floor=0.025,root=0.975,axis = 0)
    temp= temp.apply(scale,axis=0)

    temp = pd.merge(Y,temp,left_index=True,right_index=True,how="left")
    res = pd.merge(temp, info_macro,left_index=True,right_index=True,how="left").fillna(method="ffill")
    res = res.dropna(axis = 0)
    res.to_excel("data_prepocessed/"+stk_name+"_"+stk[:6]+"_稳定.xlsx")